In [121]:
import preprocess
import numpy as np
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn import metrics

In [96]:
%matplotlib osx

# Data Loading

In [3]:
%%time
# load input files
rootdir = '../../data.nosync/'
interictal_count = preprocess.count_interictal_files(rootdir)
interictal_files = preprocess.get_interictal_files(rootdir, interictal_count)
preictal_files = preprocess.get_preictal_files(rootdir)
print('Interictal file count:', len(interictal_files))
print('Preictal file count:', len(preictal_files))

Interictal file count: 480
Preictal file count: 24
CPU times: user 2.4 ms, sys: 6.01 ms, total: 8.41 ms
Wall time: 10.4 ms


In [6]:
# target vector
interictal_y = np.zeros(len(interictal_files))
preictal_y = np.ones(len(preictal_files))
y = np.concatenate((interictal_y, preictal_y), axis=0)
y.shape

(504,)

In [13]:
%%time
# input vector
X_list = []
X_list.extend(interictal_files)
X_list.extend(preictal_files)
X = np.array(X_list)
print(type(X))
X.shape

<class 'numpy.ndarray'>
CPU times: user 458 µs, sys: 262 µs, total: 720 µs
Wall time: 602 µs


(504,)

In [16]:
# summarize class distribution (0=interictal, 1=pre-ictal)
print(Counter(y))

Counter({0.0: 480, 1.0: 24})


In [20]:
# define oversampling strategy due to class imbalance
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X.reshape(-1,1), y)
print(Counter(y_over))

Counter({0.0: 480, 1.0: 480})


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.33, random_state=42)
print(X_train.shape)
print(X_test.shape)

(643, 1)
(317, 1)


# Data Visualisation

In [36]:
# load sample interictal file
x_interictal = X_train[0]
print('Label:', y_train[0])
filepath = rootdir + x_interictal.item()
print('Load file:', file_path)
mat = sio.loadmat(filepath)
sigbuf = preprocess.get_sig(mat)
fs = np.rint(preprocess.get_fs(mat))
print('Sampling rate (Hz):', fs)
print(sigbuf.shape)

Label: 0.0
Load file: ../../data.nosync/Dog_1_interictal_segment_0146.mat
Sampling rate (Hz): 400.0
(16, 239766)


In [47]:
EEG[:,0].shape

(239766,)

In [45]:
EEG.shape

(239766, 16)

In [107]:
# visualise raw signal
EEG = sigbuf.T
N = len(EEG)
t = np.arange(0,N)/fs
fig, axes = plt.subplots(figsize=(10,12), nrows=16, ncols=1, sharex=True)
for i, ax in enumerate(axes):
    ax.plot(t, EEG[:,i], color='k')
plt.xlabel('Time, s')
axes[7].set_ylabel('Amplitude, $\mu V$')
fig.tight_layout()

The lines appear 'dense', will rescaling the data help?

In [97]:
def rescale_ch(signal, smin, smax):
    signal = signal.reshape((len(signal), 1))
    scaler = MinMaxScaler(feature_range=(smin,smax))
    scaler = scaler.fit(signal)
    rescaled = scaler.transform(signal)
    rescaled = rescaled.reshape((-1,))
    return rescaled

In [98]:
# rescale data to range [0,1]
rescaled_EEG = np.empty_like(EEG)
for i in range(EEG.shape[1]):
    rescaled_EEG[:,i] = rescale_ch(EEG[:,i], smin=0, smax=1)

In [100]:
# rescale data to range [-1,1]
rescaled_EEG = np.empty_like(EEG)
for i in range(EEG.shape[1]):
    rescaled_EEG[:,i] = rescale_ch(EEG[:,i], smin=-1, smax=1)

Rescaling between [0,1] makes the signal appear flat with pulses. The same is true for range [-1,1] but this time the pulses appear in the negative axis too.

In [102]:
# rescale data to range [-10,10]
rescaled_EEG = np.empty_like(EEG)
for i in range(EEG.shape[1]):
    rescaled_EEG[:,i] = rescale_ch(EEG[:,i], smin=-10, smax=10)

Between [-10,10] the signal is less flat but it still looks abnormal compared to raw eeg as there are periods of flatness between oscillations.

In [108]:
# rescale data to range [-100,100]
rescaled_EEG = np.empty_like(EEG)
for i in range(EEG.shape[1]):
    rescaled_EEG[:,i] = rescale_ch(EEG[:,i], smin=-100, smax=100)

Rescaling is clearly impacting the signal oscillations. We shall perform experiments to determine the optimal scale when assessing knn performance.

In [105]:
# visualise rescaled signal
N = len(rescaled_EEG)
t = np.arange(0,N)/fs
fig, axes = plt.subplots(figsize=(20,12), nrows=16, ncols=1, sharex=True)
for i, ax in enumerate(axes):
    ax.plot(t, rescaled_EEG[:,i], color='k')
plt.xlabel('Time, s')
axes[7].set_ylabel('Amplitude, $\mu V$')
fig.tight_layout()

# Data Preprocessing

In [113]:
# rescale signal in range [-100,100]
X_train_EEG = np.zeros((643, 16, 239766))

for i in tqdm(range(len(X_train))):
    filepath = rootdir + X_train[i].item()
    mat = sio.loadmat(filepath)
    sigbuf = preprocess.get_sig(mat)
    rescaled_sigbuf = np.empty_like(sigbuf)
    for j in range(sigbuf.shape[0]):
        rescaled_sigbuf[j,:] = rescale_ch(sigbuf[j,:], smin=-100, smax=100)
    X_train_EEG[i] = rescaled_sigbuf

100%|██████████| 643/643 [02:50<00:00,  3.78it/s]


In [115]:
# rescale signal in range [-100,100]
X_test_EEG = np.zeros((317, 16, 239766))

for i in tqdm(range(len(X_test))):
    filepath = rootdir + X_test[i].item()
    mat = sio.loadmat(filepath)
    sigbuf = preprocess.get_sig(mat)
    rescaled_sigbuf = np.empty_like(sigbuf)
    for j in range(sigbuf.shape[0]):
        rescaled_sigbuf[j,:] = rescale_ch(sigbuf[j,:], smin=-100, smax=100)
    X_test_EEG[i] = rescaled_sigbuf

100%|██████████| 317/317 [01:28<00:00,  3.58it/s]


# Model Building

## k-Nearest Neighbour

In [120]:
X_train_EEG.shape

(643, 16, 239766)

3-dimensional input is incompatible with kNN classifier. One strategy is to reshape the input to `(643, 16*239766)` which will result in a new 2-dimensional array which preserves all the information in 3D array but results in the kNN not being able to take advantage of the implicit structure of temporal data.

In [123]:
X_train_EEG_2d = X_train_EEG.reshape(-1, 16*239766)
X_train_EEG_2d.shape

(643, 3836256)

In [128]:
%%time
k = 5
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X_train_EEG_2d, y_train)

CPU times: user 5.88 s, sys: 41.6 s, total: 47.5 s
Wall time: 2min 36s


KNeighborsClassifier()

Before evaluating the test set, we need to perform the same kind of dimensionality reduction.

In [129]:
X_test_EEG_2d = X_test_EEG.reshape(-1, 16*239766)
X_test_EEG_2d.shape

(317, 3836256)

In [ ]:
%%time
# Predict the response for test dataset
y_pred = model.predict(X_test_EEG_2d)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))